In [4]:
#GET LIST OF DATES

import GetOldTweets3 as got
import pandas as pd
import re

def getDays(start, end_date):
    
    num_days = [0,31,28,31,30,31,30,31,31,30,31,30,31]
    days = [start]

    flag = 1
    while(flag):

        s_date = re.split('-',start)
        int_year = int(s_date[0])
        int_month = int(s_date[1])
        int_day = int(s_date[2])

        #Detect new year
        if(int_month == 12 and int_day == 31):
            end = str(int_year + 1) + '-01-01'
        #Detect end of month
        elif(int_day == num_days[int_month]):

            #Add 0 to months with 1 digit, i.e 04 for April instead of just 4
            if(int_month < 9):
                end = str(int_year) + '-0' + str(int_month+1) + '-01'
            else:
                end = str(int_year) + '-' + str(int_month+1) + '-01'
        #Middle of the month
        else:
            #Add 0 to months with 1 digit, i.e 04 for April instead of just 4
            if(int_month < 10):
                #Add 0 to days with 1 digit, i.e 04 for the fourth instead of just 4
                if(int_day < 9):
                    end = str(int_year) + '-0' + str(int_month) + '-0' + str(int_day + 1)
                else:
                    end = str(int_year) + '-0' + str(int_month) + '-' + str(int_day + 1)

            else:
                if(int_day < 9):
                    end = str(int_year) + '-' + str(int_month) + '-0' + str(int_day + 1)
                else:
                    end = str(int_year) + '-' + str(int_month) + '-' + str(int_day + 1)

        days.append(end)

        start = end

        if (end == end_date):
            flag = 0
    return days


In [5]:
#CREATE DICTIONARIES FOR POSITIVE AND NEGATIVE WORDS

with open('opinion-lexicon-English/opinion-lexicon-English/positive-words.txt', 'r') as f:
    
    positive_words = {'a':[]}
    
    last_letter = 'a'
    
    for line in f:
        word = line.strip()
        
        if word[0] == last_letter:
            positive_words[last_letter].append(word)
        else:
            last_letter = word[0]
            positive_words[last_letter] = [word]
        
with open('opinion-lexicon-English/opinion-lexicon-English/negative-words.txt', 'r') as f:
    
    negative_words = {'a':[]}
    
    last_letter = 'a'
    
    for line in f:
        word = line.strip()
        
        if word[0] == last_letter:
            negative_words[last_letter].append(word)
        else:
            last_letter = word[0]
            negative_words[last_letter] = [word]

In [6]:
def sentimentAnalysis(sentence, positive_words, negative_words):
    
    sentiment_score = 0
    p_count = 0    #count of positive words
    n_count= 0    #count of negative words    
    total_words = len(sentence)
    
    try:
        for index, word in enumerate(sentence):
            word = word.lower()
            for i, v in enumerate(positive_words[word[0]]):
                if word == v:
                    p_count += 1
            for i,v in enumerate(negative_words[word[0]]):
                if word == v:
                    n_count += 1

    
    except KeyError:
        pass
    
    #RETURN STUFF
#     if p_count > n_count:
#         return 1
#     elif p_count < n_count:
#         return -1
#     else:
#         return 0

    return p_count - n_count

sentence = ['this', 'is', 'A+', 'sentence','aBorted', "abort", 'abort']


sentimentAnalysis(sentence, positive_words, negative_words)

-2

In [7]:
import json
import pandas as pd
import matplotlib.pyplot as plt
    
days = getDays('2013-04-02', '2016-03-31')

data = pd.DataFrame(columns=['Date', 'sentiment_score'])
    
for i,v in enumerate(days):
    tweets = pd.read_csv('twitter_data/' + v + '.csv', encoding = "ISO-8859-1")
    
    num_tweets = tweets['Tweet'].shape[0]
    total_sentiment = 0
    
    
    for index, value in enumerate(tweets['Tweet']):
        total_sentiment += sentimentAnalysis(value.split(), positive_words, negative_words)
    
    s_score = total_sentiment/num_tweets
    
    data = data.append({'Date' : v, 'sentiment_score' : s_score}, ignore_index=True)
    
data

#plt.plot(data['Date'], data['sentiment_score'])

#data.to_csv('datasets/twitter.csv', index=False)

,Date,sentiment_score
0,2013-04-02,0.120448
1,2013-04-03,0.163889
2,2013-04-04,0.140000
3,2013-04-05,0.125506
4,2013-04-06,0.082677
5,2013-04-07,0.023904
6,2013-04-08,0.044000
7,2013-04-09,0.008000
8,2013-04-10,0.084000
9,2013-04-11,0.064000
